# CAO Points Analysis
## Author: Ross Downey

http://www2.cao.ie/points/l8.php
***


In [1]:
# HTTP Requests Python Module
import requests as rq

# Regular expressions module
import re

# Dates and time module
import datetime as dt

# Pandas
import pandas as pd

# Downloading
import urllib.request as urlrq

<br>

## 2021 Points

***

In [2]:
# Retrieving the 2021 CAO from this URL using the requests module
resp = rq.get('http://www2.cao.ie/points/l8.php')

# Veryifing Python has downloaded the data from the URL, [200] means it's ok
resp

<Response [200]>

<br>  

## Ensuring original dataset is saved
***

In [3]:
# Saving current date and time as a function
now = dt.datetime.now()

# Convert date and time to a string
nowstr = now.strftime('%Y%m%d_%H%M%S')

In [4]:
# Filepath to be created in data folder to save original data
filepath = 'data/cao2021_' + nowstr + '.html'

<br>

## Error on Server
***
Issue with the character set defined on the CAO server.  
The server says we should decode as per:  
    Content-Type: text/html; charset=iso-8859-1  
On decoding as per 'iso-8859-1' it was noted that one line is using \x96,  
which is not defined in iso-8859-1.  
The similar decoding standard 'cp1252' will be used, which has a codepoint for \x96.

In [5]:
# Amending the original encoding
original_encoding = resp.encoding

# Changing to 'cp1252'
resp.encoding = 'cp1252'

In [6]:
# Saving the original file downloaded from server
with open(filepath, 'w') as f:
    f.write(resp.text)

In [7]:
# Compiling the regular expression for the matching lines.
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(\*?) *')
# This gives us only the lines in the original HTML code that have CAO points

<br>

## Explanation of Regular Expression
***
re.compile 
r = converting html flags to raw strings
[A-Z]{2} = Everything with 2 letters
[0-9]{3} = Followed by 3 numbers, this is each course code e.g. AL801

Two spaces followed by (.*) is space then wildcard, this is the course title string

[0-9]{3} This is the number of points required

(\*?) is a wildcard search, but some of these are actual asterisks in the html so using question mark

and asterisk at the end also as a wildcard



In [8]:
# Creating filepath to save the new csv file
filepath = 'data/cao2021_csv_' + nowstr + '.csv'


# Adding a counter to total the number of courses processed
no_lines = 0

# Opening filepath to save new csv file
with open(filepath, 'w') as f:
    
# Looping through the lines of the data 
    for line in resp.iter_lines():
        
        # Decode the line but using the 'wrong' encoding
        dline = line.decode('cp1252')
    
        # Using regular expression to match lines with courses and points only
        if re_course.fullmatch(dline):
        
            # Adding 1 for each line matched to give an overall total using the counter
            no_lines = no_lines + 1
        
            # Split the line based on number of spaces (i.e. two spaces or more) using '  +'
            linesplit = re.split('  +', dline)
        
            # Rejoin the substrings after splitting but with commas in between
            # This removes all of the spaces in the original data
            # Rejoined and using '\n' to add a new line for each course
            f.write(','.join(linesplit) + '\n')
        
# Displaying the total number of lines processed
print(f"Total number of lines is {no_lines}. ")

Total number of lines is 922. 


<br>

### Cleaning up csv file
***
1. Removed any special characters ( '#', '*' etc.) from the data to peform analysis on the
data. This was done using "Find and Replace" in Notepad ++  
2. Manually added column headers in Notepad++ to align with other years' data  
3. Manually removed commas from 35 course titles following prompts in python error messages  
4. Saved csv file as "Ver2" to read into pandas


In [115]:
df2021 = pd.read_csv('data/cao2021_csv_20211030_112255_Ver2.csv', sep=',')

In [116]:
df2021

,Course Code,Course,Rd1 Points,Rd2 Points
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
917,WD211,Creative Computing,270,NaN
918,WD212,Recreation and Sport Management,262,NaN
919,WD230,Mechanical and Manufacturing Engineering,230,230.0
920,WD231,Early Childhood Care and Education,266,NaN


In [117]:
# Spot Checking row 335, (Python index and header)
df2021.iloc[333]

Course Code                               TU814
Course         Sustainable Transport Management
Rd1 Points                                  253
Rd2 Points                                253.0
Name: 333, dtype: object

<br>

## 2020 Points
***

https://www.cao.ie/index.php?page=points&p=2020&bb=points

<br>

### Saving Original File from CAO website

***

In [ ]:
# Creating filepath for original 2020 data from CAO site
# File extension set as xlsx as original is in excel format
filepath2020 = 'data/cao2020_' + nowstr + '.xlsx'

In [ ]:
urlrq.urlretrieve('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx',
                  filepath2020)

<br>

### Loading 2020 Spreadsheet into Notebook using Pandas

***

In [29]:
# Downloading and Parsing the spreadsheet
# Skipping initial 10 rows as no relevant data contained within
df2020 = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx',
                       skiprows=10,  usecols = [1,2,3,7,9])
df2020.rename(columns={'COURSE TITLE': 'Course', 'COURSE CODE2': 'Course Code',
                      'EOS Mid-point' : 'Mid', 'R1 POINTS' : 'R1 Points'},
              inplace=True)
# Re-labelling columns to align with other years' data
# https://stackoverflow.com/questions/11346283/renaming-column-names-in-pandas
df2020=df2020[['Course Code', 'Course', 'R1 Points', 'EOS', 'Mid']]
# Re-arranging columns to align with other years' data
# https://erikrood.com/Python_References/change_order_dataframe_columns_final.html

In [30]:
# Checking first five rows for formatting
df2020.head()

,Course Code,Course,R1 Points,EOS,Mid
0,AC120,International Business,209,209,280
1,AC137,Liberal Arts,252,252,270
2,AD101,"First Year Art & Design (Common Entry,portfolio)",#+matric,#+matric,#+matric
3,AD102,Graphic Design and Moving Image Design (portfo...,#+matric,#+matric,#+matric
4,AD103,Textile & Surface Design and Jewellery & Objec...,#+matric,#+matric,#+matric


In [34]:
df2020.tail()

,Course Code,Course,R1 Points,EOS,Mid
1459,WD208,Manufacturing Engineering,188,188,339
1460,WD210,Software Systems Development,279,279,337
1461,WD211,Creative Computing,271,271,318
1462,WD212,Recreation and Sport Management,270,270,349
1463,WD230,Mechanical and Manufacturing Engineering,253,253,369


In [18]:
# Spot checking random row
# Index 333 is the same as row 345 in original excel file
# Removed 10 rows, 1 row for header and 1 row for pandas index starting at 0
# Spot check aligns with original excel file
df2020.iloc[333]

CATEGORY (i.e.ISCED description)                       Arts
COURSE TITLE                        Film and Creative Media
COURSE CODE2                                          DB576
R1 POINTS                                               AQA
R1 Random *                                             NaN
R2 POINTS                                               AQA
R2 Random*                                              NaN
EOS                                                     AQA
EOS Random *                                            NaN
EOS Mid-point                                           338
LEVEL                                                     7
HEI                                  Dublin Business School
Test/Interview #                                        NaN
avp                                                     avp
v                                                       NaN
Column1                                                 NaN
Column2                                 

In [ ]:
# Spot Check last row
# Aligns with original excel file
df2020.iloc[-1]

In [ ]:
# Creating file path for 2020 data in csv format (Note file extension)
filepath2020 = 'data/cao2020_' + nowstr + '.csv'

In [ ]:
# Saving dataframe
df2020.to_csv(filepath2020)

<br>

## 2019 Points
***

https://www.cao.ie/index.php?page=points&p=2019&bb=points

Steps taken to convert original pdf file to pandas dataframe
<br>
1. Original PDF file downloaded from link above
2. PDF file converted to Microsoft Word file
3. Microsoft Word file saved in docx format
4. Second copy of Word file saved as "_edited"
5. Headers, Footers and initial introduction deleted
6. Selected all and pasted into Notepad++
7. Removed all HEI names as had their own rows
8. Removed all special characters (#* etc.) using "Find and Replace"
9. Changed backticks (`) to apostrophes (') using "Find and Replace"
10. Changed column header from "Course and Institution" to "Course"
11. Extra "NaN" columns noted, removed double tab from course code AL870 (l.28)
12. Extra tabs at end of rows removed as not required.


In [14]:
df2019 = pd.read_csv('data/cao2019_20211102_194800_edited.csv', sep='\t',
                    usecols = [0,1,2,3])
# Ref https://www.listendata.com/2019/06/pandas-read-csv.html

In [15]:
df2019

,Course Code,Course,EOS,Mid
0,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,AL802,Software Design with Cloud Computing,301,306.0
2,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,AL805,Network Management and Cloud Infrastructure,329,442.0
4,AL810,Quantity Surveying,307,349.0
...,...,...,...,...
925,WD200,Arts (options),221,296.0
926,WD210,Software Systems Development,271,329.0
927,WD211,Creative Computing,275,322.0
928,WD212,Recreation and Sport Management,274,311.0


***
## End